In [ ]:
import sys
import os

sys.path.append(os.path.abspath("../.."))

from tiny_graph.graph.executable import Graph
from tiny_graph.models.base import GraphState
from tiny_graph.buffer.factory import History, LastValue
from tiny_graph.checkpoint.storage.local_storage import LocalStorage
from tiny_graph.constants import START, END
import time
from typing import Dict
from rich import print as rprint


# Define our state model
class ProcessState(GraphState):
    status: LastValue[str]
    results: History[Dict[str, float]]


# Initialize state and graph with local storage and chain_id
chain_id = "process_workflow_v1"
state = ProcessState(status="", results={})
graph = Graph(state=state, checkpoint_storage=LocalStorage)


# Define processing nodes
@graph.node()
def initialize_process(state):
    time.sleep(0.5)  # Simulate work
    return {"status": "initializing"}


@graph.node()
def process_data_1(state):
    time.sleep(0.5)  # Simulate work
    return {"status": "processing_1", "results": {"accuracy": 0.85, "step": 1.0}}


@graph.node()
def process_data_2(state):
    time.sleep(0.5)  # Simulate work
    return {"status": "processing_2", "results": {"accuracy": 0.92, "step": 2.0}}


@graph.node()
def finalize(state):
    time.sleep(0.5)  # Simulate work
    return {"status": "completed"}


# Create the workflow
graph.add_edge(START, "initialize_process")
graph.add_edge("initialize_process", "process_data_1")
graph.add_edge("process_data_1", "process_data_2")
graph.add_edge("process_data_2", "finalize")
graph.add_edge("finalize", END)

# Compile and execute
graph.compile()


def run_workflow():
    print("Starting execution...")
    print(f"Using chain_id: {graph.chain_id}")

    # Execute the graph (checkpoints will be automatically saved)
    graph.execute()

    print("\nExecution completed!")
    print(f"Final status: {graph.state.status}")
    print(f"Results history: {graph.state.results}")

    print("\nVerifying final state from checkpoint:")
    rprint(f"Checkpoints: {graph.checkpoint_storage._storage}")


# Run the example
if __name__ == "__main__":
    run_workflow()

In [ ]:
graph.execution_plan

In [ ]:
graph.checkpoint_storage.chain_id

In [ ]:
state.version

In [ ]:
sorted_checks = sorted(
    graph.checkpoint_storage._storage[graph.chain_id].values(),
    key=lambda x: x["timestamp"],
)

if sorted_checks:
    print("x")

In [ ]:
graph.checkpoint_storage._storage.get(graph.chain_id, None)

In [ ]:
print(graph.checkpoint_storage._get_last_stored_model_version(chain_id))

In [ ]:
class TestState(GraphState):
    value: LastValue[int]
    text: History[list[str]]


# Initialize
state = TestState(value=42, text="asdf")
graph = Graph(state=state, checkpoint_storage=LocalStorage)

# Save checkpoint
checkpoint_id = graph.checkpoint_storage.save_checkpoint(state, graph.chain_id)

# Load checkpoint
loaded_state = graph.checkpoint_storage.load_checkpoint(
    state, graph.chain_id, checkpoint_id
)